In [1]:
import sys


no_of_cpu = 4
max_cores = 4
executor_mem = '26g'



Job_Name = 'Abhimanyu: DQ scripts'

from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = ''
es_pwd = ''
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
17200,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.functions import hash
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2024-01-19 12:49:57.596547'

In [3]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
###latest_progress=get_latest_progress(table_name,source)
# dates in string format
# dates in string format
str_d1 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")-datetime.timedelta(days=30)
d2 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-2#currently T-2 change 2 to 1 for T-1

time_filter={'start_date':d1,'end_date':d2+datetime.timedelta(days=day)}
time_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'start_date': datetime.datetime(2023, 12, 20, 0, 0), 'end_date': datetime.datetime(2024, 2, 16, 0, 0)}

In [10]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"public",
         "dbtable": "public.policy_gc_gen_prop_information_tab",
         "server.port":"1150-1170"} 

#business_sourceinfo

df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
    .filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
business_sourceinfo_count= df.filter(col("txt_business_source_info").rlike("'%`~!@#$%^&*()-_=+[{]}|;:,<.>/?%';")).count()
business_sourceinfo_count_full=df.count()
print(business_sourceinfo_count)
print(business_sourceinfo_count_full)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0
217214192

In [12]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"public",
         "dbtable": "public.policy_dh_risk_headers_mot",
         "server.port":"1150-1170"} 

# source_system

df1 = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
    .filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
source_system_count= df1.filter(col("source_system").rlike("'%`~!@#$%^&*()-_=+[{]}|;:,<.>/?%';")).count()  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
count_source_system_full=df1.count()
print(source_system_count)
print(count_source_system_full)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0
0
145343602

In [20]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"registers",
         "dbtable": "registers.premium_register",
         "server.port":"1150-1170"} 

# client_name

df2 = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
    .filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
client_name_count = df2.filter(col("client_name").rlike("'%`~!@#$%^&*()-_=+[{]}|;:,<.>/?%';")).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o209.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 (TID 13, insightsuat.tataaig.com/agent_4, executor 2): ExecutorLostFailure (executor 2 exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 630856 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1925)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1913)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1912)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1912)
	at org

In [18]:
client_name_count_full=df2.count()
print(client_name_count)
print(count_source_system_full)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [ ]:
#client_state

In [21]:
client_state_count = df2.filter(col("client_state").rlike("'%`~!@#$%^&*()-_=+[{]}|;:,<.>/?%';")).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
client_state_count_full=df2.count()
print(client_state_count)
print(client_state_count_full)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0
317017885

In [ ]:
#cust_res_city

In [23]:
cust_res_city_count = df2.filter(col("cust_res_city").rlike("'%`~!@#$%^&*()-_=+[{]}|;:,<.>/?%';")).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
cust_res_city_count_full=df2.count()
print(cust_res_city_count)
print(cust_res_city_count_full)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#zone

In [ ]:
zone_count = df2.filter(col("zone").rlike("'%`~!@#$%^&*()-_=+[{]}|;:,<.>/?%';")).count()

In [ ]:
zone_count_full=df2.count()
print(zone_count)
print(zone_count_full)

In [ ]:
#record_type_desc

In [ ]:
distinct_record_types_count = ( df2.filter(df2.report_accounting_period == '2024-01-31')
				.select("record_type_desc")
				.distinct()

In [ ]:
distinct_record_types_count_full=df2.count()
print(distinct_record_types_count)
print(distinct_record_types_count_full)

In [ ]:
#policy_counter

In [ ]:
distinct_policy_counter_count = ( df2.filter(df2.report_accounting_period == '2024-01-31')
				.select("policy_counter")
				.distinct()

In [ ]:
distinct_policy_counter_count_full=df2.count()
print(distinct_policy_counter_count)
print(distinct_policy_counter_count_full)

In [ ]:
#branch

In [ ]:
distinct_branch_count = ( df2.filter(df.report_accounting_period == '2024-01-31')
				.select("branch")
				.distinct()

In [ ]:
distinct_branch_count_full=df2.count()
print(distinct_branch_count)
print(distinct_branch_count_full)

In [ ]:
#Commission_Percentage

In [ ]:
distinct_Commission_Percentage_count = ( df2.filter(df2.report_accounting_period == '2024-01-31')
				.select("Commission_Percentage")
				.distinct()

In [ ]:
distinct_Commission_Percentage_count_full=df2.count()
print(distinct_Commission_Percentage_count)
print(distinct_Commission_Percentage_count_full)

In [ ]:
#no_of_life_covered

In [ ]:
distinct_no_of_life_covered_count = ( df2.filter(df2.report_accounting_period == '2024-01-31')
				.select("no_of_life_covered")
				.distinct()

In [ ]:
distinct_no_of_life_covered_count_full=df2.count()
print(distinct_no_of_life_covered_count)
print(distinct_no_of_life_covered_count_full)

In [ ]:
#Premium_amount_without_inr

In [ ]:
distinct_Premium_amount_without_inr_count = ( df2.filter(df2.report_accounting_period == '2024-01-31')
				.select("Premium_amount_without_inr")
				.distinct()

In [ ]:
distinct_Premium_amount_without_inr_count_full=df2.count()
print(distinct_Premium_amount_without_inr_count)
print(distinct_Premium_amount_without_inr_count_full)

In [ ]:
#ri_inward_flag

In [ ]:
distinct_ri_inward_flag_count = ( df2.filter(df2.report_accounting_period == '2024-01-31')
				.select("distinct_ri_inward_flag")
				.distinct()

In [ ]:
distinct_ri_inward_flag_count_full=df2.count()
print(distinct_ri_inward_flag_count)
print(distinct_ri_inward_flag_count_full)

In [ ]:
#chennal

In [ ]:
distinct_Channel_count = ( df.filter(df.report_accounting_period == '2024-01-31')
				.select("Channel")
				.distinct()

In [ ]:
distinct_ri_inward_flag_count_full=df2.count()
print(distinct_Channel_count)
print(distinct_Channel_count_full)

In [ ]:
#sgst

In [ ]:
sgst_count = df2.filter((df2.report_accounting_period == '2024-01-31') & (df2.sgst == 0)).select("sgst","sgst").distinct()

In [ ]:
distinct_sgst_count_full=df2.count()
print(distinct_sgst_count)
print(distinct_sgst_count_full)

In [ ]:
#igst

In [ ]:
igst_count = df2.filter((df2.report_accounting_period == '2024-01-31') & (df2.igst == 0)).select("igst","igst").distinct()

In [ ]:
distinct_igst_count_full=df2.count()
print(distinct_igst_count)
print(distinct_igst_count_full)

In [ ]:
#cgst

In [ ]:
cgst_count = df2.filter((df.report_accounting_period == '2024-01-31') & (df2.cgst == 0)).select("cgst","cgst").distinct()

In [ ]:
distinct_cgst_count_full=df2.count()
print(distinct_cgst_count)
print(distinct_cgst_count_full)

In [ ]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"public",
         "dbtable": "policy_dh_risk_headers_hlt",
         "server.port":"1150-1170"} 



df3 = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
    .filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))

In [ ]:
issuance_mode_count = df3.filter(col("issuance_mode").rlike("'%`~!@#$%^&*()-_=+[{]}|;:,<.>/?%';")).count()

In [ ]:
cust_res_city_count_full=df3.count()
print(issuance_mode)
print(issuance_mode_full)

In [ ]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"registers",
         "dbtable": "registers.premium_register",
         "server.port":"1150-1170"} 

# Get the data  from credit_bureau_data Table

df2 = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
    .filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))